# Step 0. 데이터 정제 전 '전체 점검' (EDA 이전)
- 작성일: 2026-01-27  
- 목표: **정제를 시작하기 전에 데이터 상태를 빠르게 확인**하고, EDA/모델링에서 터질 포인트를 미리 잡아둔다.  
- 원칙: 이 노트북에서는 **데이터를 수정/저장하지 않는다(확인용)**..

<details>
<summary><strong>0. 데이터 로드 (data/raw)</strong></summary>

- 데이터는 `data/raw`에 있다고 가정한다.  
- 코드가 길어지지 않게 **상대경로**로만 읽는다.

<details>
<summary><em>인사이트(메모)</em></summary>

- 이후 노트북에서도 동일한 경로를 쓰기 위해 `DATA_DIR`만 고정한다.

</details>
</details>

In [2]:
from pathlib import Path
import pandas as pd

BASE_DIR = Path.cwd().parents[2]   # vote-AI
DATA_DIR = BASE_DIR / "data" / "raw"

train = pd.read_csv(DATA_DIR / "train.csv")
test  = pd.read_csv(DATA_DIR / "test_x.csv")
sub   = pd.read_csv(DATA_DIR / "sample_submission.csv")

print(train.shape, test.shape, sub.shape)

(45532, 78) (11383, 77) (11383, 2)


<details>
<summary><strong>1. 스키마(컬럼) 검증</strong></summary>

- train에는 타깃 `voted`가 있어야 한다.
- test에는 `voted`가 없어야 한다.
- 제출 형식(sample_submission)의 컬럼을 확인한다.

<details>
<summary><em>인사이트(메모)</em></summary>

- 스키마가 꼬이면 이후 모든 분석이 의미가 없어짐 → 가장 먼저 확인.
- `index`는 식별자(모델 입력 제외 예정).

</details>
</details>

In [3]:
assert "voted" in train.columns, "train에 voted(타깃) 컬럼이 없습니다."
assert "voted" not in test.columns, "test에 voted가 있으면 안 됩니다."

print("train columns:", len(train.columns))
print("test  columns :", len(test.columns))
print("submission columns:", sub.columns.tolist())

train columns: 78
test  columns : 77
submission columns: ['index', 'voted']


<details>
<summary><strong>2. 타깃(`voted`) 분포 확인 + 인코딩 계획</strong></summary>

- 코드북 기준: `voted`는 1=Yes(투표), 2=No(비투표)
- 모델링 단계에서는 보통 0/1이 편하므로 아래처럼 변환을 계획한다.
  - `y = (voted == 1).astype(int)`  → 1=투표, 0=비투표

<details>
<summary><em>인사이트(메모)</em></summary>

- 1 = 20634명, 2 = 24898명 
- 투표 45% 비투표 54% 

</details>
</details>

In [4]:
vc = train["voted"].value_counts(dropna=False).sort_index()
print(vc)
print("\nclass ratio(%):")
print((vc / vc.sum() * 100).round(2))
print("\nunique:", sorted(train["voted"].unique()))

voted
1    20634
2    24898
Name: count, dtype: int64

class ratio(%):
voted
1    45.32
2    54.68
Name: count, dtype: float64

unique: [np.int64(1), np.int64(2)]


<details>
<summary><strong>3. 식별자/중복 점검</strong></summary>

- `index` 유일성 확인(식별자 역할)
- 완전 중복 행(duplicate row) 존재 여부 확인

<details>
<summary><em>인사이트(메모)</em></summary>

- 중복 행이 많으면 학습이 왜곡될 수 있음(특히 train).  
- 이번 데이터에서는 `index`가 0부터 연속인 형태로 보였고, 중복은 거의 없었다.

</details>
</details>

In [5]:
if "index" in train.columns:
    print("train index unique?:", train["index"].is_unique)
    print("test  index unique?:", test["index"].is_unique)
    print("train index min/max:", train["index"].min(), train["index"].max())
    print("test  index min/max:", test["index"].min(), test["index"].max())
else:
    print("No `index` column in train/test.")

print("duplicate rows - train:", train.duplicated().sum())
print("duplicate rows - test :", test.duplicated().sum())

train index unique?: True
test  index unique?: True
train index min/max: 0 45531
test  index min/max: 0 11382
duplicate rows - train: 0
duplicate rows - test : 0


<details>
<summary><strong>4. NaN 결측치 점검</strong></summary>

- `isna().mean()`은 컬럼별 NaN 비율을 계산한다.
- NaN이 0%여도, 설문 데이터는 **코드값(0 등)**으로 결측/무응답을 표현할 수 있으니 다음 단계에서 점검해보기로 한다.

<details>
<summary><em>인사이트(메모)</em></summary>

- 이번 데이터는 NaN이 거의 없었다.  
- 그래서 “NaN 결측”보다 “코드 결측/척도 위반/이상치”를 확인해 보아야한다.

</details>
</details>

In [ ]:
nan_train = train.isna().mean().sort_values(ascending=False)
nan_test  = test.isna().mean().sort_values(ascending=False)

print("Top NaN ratio (train):")
print((nan_train.head(10)*100).round(3))

print("\nTop NaN ratio (test):")
print((nan_test.head(10)*100).round(3))

<details>
<summary><strong>5. 코드값(0) 비율 점검 (의심 포인트 표시)</strong></summary>

- 숫자형 컬럼에서 `0` 비율이 높은 컬럼을 뽑는다.
- 목적: “바로 처리”가 아니라 **의심 컬럼 리스트업**.

<details>
<summary><em>인사이트(메모)</em></summary>

- `wr_*`, `wf_*`는 이진 변수라서 0이 많아도 정상(=No).  
- `education/urban/hand`의 0은 무응답 코드일 가능성.  
- `tp*`는 코드북(1~5)인데 0이 섞여 “척도/무응답 코드” 이슈로 이어졌었다.

</details>
</details>

In [ ]:
def top_code_ratio(df, code_value=0, topk=30):
    ratios = {}
    for c in df.columns:
        if c == "voted":
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            ratios[c] = (df[c] == code_value).mean()
    return pd.Series(ratios).sort_values(ascending=False).head(topk)

zero_ratio = top_code_ratio(train, code_value=0, topk=30)
print("Top columns by zero ratio (train):")
print((zero_ratio*100).round(2))

<details>
<summary><strong>6. 분포 이상치(꼬리/극단값) 점검</strong></summary>

- 숫자형 컬럼별 상위값(top 5)을 직접 확인한다.
- “오류값” vs “행동 기반 극단치(응답시간)”를 구분하는 용도.

<details>
<summary><em>인사이트(메모)</em></summary>

- `familysize`: 2147483647 / 999 / 100 등 **확정 오류값** 발견  
- `Q*E`: 수천만 단위 **극단치** 존재(방치/이탈 응답 추정)  
- `tp*`: 0/6/7 값이 많아 **척도 정의 문제**로 분리해서 봐야 함

</details>
</details>

In [6]:
num_cols = train.select_dtypes(include=[np.number]).columns
for c in num_cols:
    top = train[c].sort_values(ascending=False).head(5).values
    print(f"\n[{c}]")
    print(top)


[index]
[45531 45530 45529 45528 45527]

[QaA]
[5. 5. 5. 5. 5.]

[QaE]
[2413960  697050  599190  571768  497807]

[QbA]
[5. 5. 5. 5. 5.]

[QbE]
[5580395 3232042 2201314 1067553  920737]

[QcA]
[5. 5. 5. 5. 5.]

[QcE]
[871557 625243 557184 537531 523947]

[QdA]
[5. 5. 5. 5. 5.]

[QdE]
[1552821  893392  762715  523757  507927]

[QeA]
[5. 5. 5. 5. 5.]

[QeE]
[1919926 1790548 1401417 1038584  793900]

[QfA]
[5. 5. 5. 5. 5.]

[QfE]
[11763701  8953454  4469148  4170563  2153693]

[QgA]
[5. 5. 5. 5. 5.]

[QgE]
[1068252  723197  669341  631259  564700]

[QhA]
[5. 5. 5. 5. 5.]

[QhE]
[107086013   1856528   1178668    691158    581619]

[QiA]
[5. 5. 5. 5. 5.]

[QiE]
[17035860 11524149 10300010  4817813  4648542]

[QjA]
[5. 5. 5. 5. 5.]

[QjE]
[9910006 4742410 2392799 1213936 1058971]

[QkA]
[5. 5. 5. 5. 5.]

[QkE]
[4824953 3098699 1829652 1665055 1414246]

[QlA]
[5. 5. 5. 5. 5.]

[QlE]
[77543511  8264414  4146511  2853554  1887223]

[QmA]
[5. 5. 5. 5. 5.]

[QmE]
[14108320  5454511  4921238  149

<details>
<summary><strong>6-1. (핵심) 이상치 후보 3종 집중 확인</strong></summary>

- `familysize` : 비현실 값/오류값 확인  
- `Q*E` : 상위 꼬리(극단치) 규모 확인  
- `tp*` : 값 범위(0/6/7) 분포 확인  

<details>
<summary><em>인사이트(메모)</em></summary>

- `familysize > 20`이 소수 존재했고, max가 2147483647로 오류값 가능성이 높았음  
- `Q*E`는 q99 대비 max 비율이 매우 커서 클리핑/로그 후보  
- `tp*`는 0/6/7이 대량이라 “단순 이상치”가 아니라 “척도/무응답 코드”로 다루는 게 자연스러움

</details>
</details>

In [ ]:
# familysize: 상식 범위를 넘어가는 값 확인(예: >20)
if "familysize" in train.columns:
    print("familysize > 20 count:", (train["familysize"] > 20).sum())
    print(train.loc[train["familysize"] > 20, "familysize"].describe())

# Q*E: q99 초과 개수(상위 1%)는 원래 1%지만, 값의 스케일을 확인
E_cols = [c for c in train.columns if c.endswith("E")]
if len(E_cols) > 0:
    sample_E = E_cols[:5]  # 너무 길어지지 않게 일부만
    for c in sample_E:
        q99 = train[c].quantile(0.99)
        print(f"{c} > q99 count:", (train[c] > q99).sum(), "/", len(train), "| q99:", round(q99, 2), "| max:", int(train[c].max()))

# tp 값 분포 (0/6/7 여부)
tp_cols = [c for c in train.columns if c.startswith("tp")]
if len(tp_cols) > 0:
    for c in tp_cols:
        vc = train[c].value_counts(dropna=False).sort_index()
        # 너무 길어질 수 있어 dict 형태로 출력
        print(c, vc.to_dict())

<details>
<summary><strong>7. Step 0 결론 (전처리 방향에 대한 최종 정리)</strong></summary>

Step 0에서의 목적은  
**“가설 혹은 데이터의 결측치,이상치를 확인하기 위한 과정”**이다.

이번 데이터 점검을 통해 다음과 같은 특징을 확인했다.

---

## 7-1. familysize (가족 구성원 수)

### 관찰된 사실
- 대부분 값은 1~10명 내외
- 그런데 일부 샘플에서  
  - 999  
  - 2147483647 (비현실적인 매우 큰 수)  
  같은 값이 존재

### 해석
- 현실적인 가족 수로 보기 어려움
- 입력 오류, 무응답 코드, 시스템 값이 섞였을 가능성이 큼
- 그대로 두면 모델이 **“가족 수가 매우 크면 투표 성향이 다르다”**고 잘못 학습할 위험이 있음

### 합리적인 처리 방향
- 가족 수가 **상식적으로 불가능한 값(예: 20 초과)** 인 경우  
  → **결측치(NaN)로 바꾼 뒤 평균/중앙값으로 채우는 방식**이 가장 무난하다 생각한다.

---

## 7-2. Q*E (질문 응답 시간 관련 변수들)

### 관찰된 사실
- 대부분의 응답 시간은 수천 단위
- 하지만 일부 샘플은 수백만~수천만 단위로 튀는 값이 존재
- 상위 1% 값이 나머지 분포와 완전히 분리됨

### 해석
- 실제 응답 시간이라기보다는  
  - 설문 중단  
  - 브라우저 방치  
  - 시스템 기록 오류  
  가능성이 높을것이라 생각한다.
- 이런 값은 **“극단적인 꼬리값”**으로, 평균 기반 모델에 큰 영향을 줄것이라 본다

### 처리 방향
1. **상위 1%보다 큰 값은 잘라내기**
   - 예: “상위 99% 값까지만 인정”
2. 그 후 선택지
   - 그대로 사용
   - 또는 로그 변환(log1p)으로 스케일을 완만하게 만들어 보기


---

## 7-3. tp* (성격 척도 변수)

### 관찰된 사실
- 코드북상 tp는 보통 1~5 또는 1~7 척도
- 실제 데이터에는 0, 6, 7 값이 함께 존재
- 특히 0의 비율이 꽤 큼

### 해석
- 0은 **무응답/모름**일 가능성이 높음
- 6, 7은:
  - 확장 척도일 수도 있고
  - 극단 응답일 수도 있음
- 이 부분은 “확인해 봐야 할것같다"

### 처리 방향
- **A안**  
  - 0, 6, 7 → 결측치로 처리  
  - 1~5만 사용  
  → 해석이 깔끔하고 안정적인 모델

- **B안**  
  - 0만 결측치  
  - 1~7 전부 사용  
  → 정보는 많지만 노이즈 가능성 있음

→ 두 안을 **실제로 비교 실험**해서 성능으로 판단하는 것이 가장 합리적이 생각함

---

## 7-4. 타깃 변수 voted 인코딩

### 관찰된 사실
- voted:  
  - 1 = 투표함  
  - 2 = 투표 안 함

### 계획
- 모델 학습에서는 0/1이 직관적이므로
```python
y = (voted == 1).astype(int)

# 📘 변수 설명서 (Codebook)

본 데이터는 설문 응답 기반의 정형 데이터로,  
개인 특성·성향·지식 수준이 **지난 해 국가 선거 투표 여부(voted)**에  
어떤 영향을 미치는지 분석하는 것을 목표로 한다.

## 1. 기본 식별자

### `index`
- 각 응답자의 고유 식별자
- 연속적이며 중복 없음
- **모델 입력에는 사용하지 않음**
- 제출 파일 정렬 및 식별용으로만 사용

---

## 2. 태도/가치관 질문 (Q_A, Q_E)

### `Qa ~ Qt`
- 사회적 신뢰, 도덕성, 인간관, 현실 인식 등을 묻는 문항
- 일부 문항은 **비식별화(Secret)** 처리됨

#### 응답 척도 (순서형)
| 값 | 의미 |
|--|--|
| 1 | Disagree |
| 2 | Slightly disagree |
| 3 | Neutral |
| 4 | Slightly agree |
| 5 | Agree |

📌 **모델링 관점**
- 순서형 변수
- 트리 모델에서는 그대로 사용 가능
- 선형 모델에서는 스케일링 또는 ordinal encoding 고려

---

### `Q_Ea ~ Q_Et`
- 각 질문에 **응답하는 데 걸린 시간**
- 반응 속도 / 인지적 부담 / 성실성 지표 가능성

📌 **모델링 관점**
- 이상치(outlier) 존재 가능 → 분포 확인 필요
- 로그 변환 고려 가능

---

## 3. 인구통계 변수

### `age_group`
- 연령대 구분 변수
- 순서형

---

### `education` (교육 수준)
| 값 | 의미 |
|--|--|
| 1 | Less than high school |
| 2 | High school |
| 3 | University degree |
| 4 | Graduate degree |
| 0 | 무응답 |

📌 0은 **결측이 아닌 무응답 코드**

---

### `engnat` (모국어가 영어인지)
| 값 | 의미 |
|--|--|
| 1 | Yes |
| 2 | No |
| 0 | 무응답 |

---

### `familysize`
- 형제자매 수
- 수치형

---

### `gender`
- Male / Female
- 범주형

---

### `hand` (주 사용 손)
| 값 | 의미 |
|--|--|
| 1 | Right |
| 2 | Left |
| 3 | Both |
| 0 | 무응답 |

---

### `married` (혼인 상태)
| 값 | 의미 |
|--|--|
| 1 | Never married |
| 2 | Currently married |
| 3 | Previously married |
| 0 | Other |

---

### `race` (인종)
- Asian, Arab, Black, Indigenous Australian,
  Native American, White, Other

---

### `religion` (종교)
- Agnostic, Atheist, Buddhist,
  Christian_Catholic, Christian_Mormon,
  Christian_Protestant, Christian_Other,
  Hindu, Jewish, Muslim, Sikh, Other

---

### `urban` (유년기 거주 환경)
| 값 | 의미 |
|--|--|
| 1 | Rural |
| 2 | Suburban |
| 3 | Urban |
| 0 | 무응답 |

---

## 4. 성격 특성 (Big Five 기반)

### `tp01 ~ tp10`
“I see myself as …”에 대한 자기 평가

| 변수 | 설명 |
|--|--|
| tp01 | Extraverted, enthusiastic |
| tp02 | Critical, quarrelsome |
| tp03 | Dependable, self-disciplined |
| tp04 | Anxious, easily upset |
| tp05 | Open to new experiences |
| tp06 | Reserved, quiet |
| tp07 | Sympathetic, warm |
| tp08 | Disorganized, careless |
| tp09 | Calm, emotionally stable |
| tp10 | Conventional, uncreative |

- **순서형 변수**
- 0은 사용되지 않음 (모든 응답은 1~5)

📌 **모델링 관점**
- 그대로 사용 권장
- 필요 시 표준화 가능

---

## 5. 단어 인지 테스트 (중요)

### `wr_01 ~ wr_13`
- **실존하는 단어**의 정의를 알고 있는지
| 값 | 의미 |
|--|--|
| 1 | Yes |
| 0 | No |

→ **지식 수준 / 어휘력 지표**

---

### `wf_01 ~ wf_03`
- **허구(가짜) 단어**의 정의를 안다고 응답했는지
| 값 | 의미 |
|--|--|
| 1 | Yes |
| 0 | No |

📌 **중요 해석**
- `wf_* == 1` : 존재하지 않는 단어를 안다고 응답 → **과장/허위 응답 성향**
- `wf_* == 0` : 정상적인 “모름” 응답  
  → **결측 아님**

📌 **모델링 활용**
- 그대로 사용
- 또는 합성 지표:
```python
wf_sum = wf_01 + wf_02 + wf_03